# Running on new images
This notebook will walk you step by step through the process of using a pre-trained model to detect traffic signs in an image.

# Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import os
import tensorflow as tf

# tf.enable_eager_execution()

from matplotlib import pyplot as plt
# from PIL import Image
import glob as glob
import random
import pickle

import sys

%matplotlib inline
# tf.executing_eagerly()

# Environment setup

In [2]:


sys.path.append('../models/research')  # Replace with the path to TensorFlow Object Detection API
sys.path.append('../darkflow')  # Replace with the path to Darkflow

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


# Tensorflow Object Detection API

## Model preparation 

In [3]:
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous'
# MODEL_NAME = 'faster_rcnn_resnet_101'
# MODEL_NAME = 'faster_rcnn_resnet50'
MODEL_NAME = 'faster_rcnn_inception_v2'
# MODEL_NAME = 'rfcn_resnet101'
# MODEL_NAME = 'ssd_inception_v2'
# MODEL_NAME = 'ssd_mobilenet_v1'

In [4]:
# Path to frozen detection graph. This is the actual model that is used for the traffic sign detection.
MODEL_PATH = os.path.join('models', MODEL_NAME)
PATH_TO_CKPT = os.path.join(MODEL_PATH,'inference_graph/frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('gtsdb_data', 'gtsdb3_label_map.pbtxt')

NUM_CLASSES = 3


## Load a (frozen) Tensorflow model into memory

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `2`, we know that this corresponds to `mandatory`.

In [6]:
PATH_TO_LABELS = '/home/yuxing/my_project/traffic-sign-detection/gtsdb_data/gtsdb3_label_map.pbtxt'

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(label_map)



item {
  name: "prohibitory"
  id: 1
}
item {
  name: "mandatory"
  id: 2
}
item {
  name: "danger"
  id: 3
}



## Helper code

In [7]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    #return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
    return np.array(image.getdata()).reshape((im_height, im_width, 3))

## Detection

In [8]:

PATH_TO_TEST_IMAGES_DIR = '/home/yuxing/my_project/traffic-sign-detection/test_images'

TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))

# Size, in inches, of the output images.
IMAGE_SIZE = (20, 20)

In [9]:

saved_detection_results = {}

# Run inference and visualization
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            image_np_expanded = np.expand_dims(image_np, axis=0)

            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})

            saved_detection_results[image_path] = {
                'boxes': boxes,
                'scores': scores,
                'classes': classes,
                'num_detections': num_detections
            }

            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=6)
            plt.figure(idx, figsize=IMAGE_SIZE)
            plt.axis('off')
            plt.imshow(image_np)


In [10]:

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            
            image_data = tf.read_file(image_path)
            image_decoded = tf.image.decode_image(image_data)
            image_np = sess.run(image_decoded)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            
            print("Shape of boxes:", boxes.shape)
            print("Shape of scores:", scores.shape)
            print("Shape of classes:", classes.shape)
            print("Shape of num_detections:", num_detections.shape)

            
            # Take the detection with the highest score
            max_score_index = np.argmax(scores)
            target_class = int(classes[0][max_score_index])
            target_score = scores[0][max_score_index]
            
            # Inverting the scores to approximate logits (this is an approximation)
            approx_logits = np.log(target_score / (1 - target_score))
        
            # Adjust target_class index
            adjusted_target_class = target_class - 1  # Now it should be between 0 and NUM_CLASSES - 1

            # Create a tensor for logits
            logits_tensor = np.zeros(NUM_CLASSES)

            # Now adjusted_target_class will be within the bounds
            logits_tensor[adjusted_target_class] = approx_logits
            
            # Define the loss after knowing target class
            target_label = tf.placeholder(tf.int32, shape=(), name='target_label')
            
            loss_object = tf.losses.softmax_cross_entropy(
                onehot_labels=tf.one_hot([target_label], NUM_CLASSES),
                logits=logits_tensor  # Changed this line
            )
            
            grads = tf.gradients(loss_object, image_tensor)[0]  # Adding [0] to get the first element
            
            # Print target class and gradient tensor
            print(f"Target Class: {target_class}")
            print(f"Gradient tensor: {grads}")
            
            if grads is None:
                print("Gradient is None.")
                continue
            
            feed_dict = {
                image_tensor: image_np_expanded,
                target_label: target_class
            }
            
            adv_grads_val = sess.run(grads, feed_dict=feed_dict)
            
            # Create adversarial image
            eps = 0.3
            adv_image = image_np + eps * np.sign(adv_grads_val)
            adv_image = np.clip(adv_image, 0, 255).astype(np.uint8)

            # Visualize the adversarial image
            plt.figure(idx, figsize=IMAGE_SIZE)
            plt.axis('off')
            plt.imshow(adv_image[0])
            plt.show()


Shape of boxes: (1, 300, 4)
Shape of scores: (1, 300)
Shape of classes: (1, 300)
Shape of num_detections: (1,)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Target Class: 3
Gradient tensor: None
Gradient is None.
Shape of boxes: (1, 300, 4)
Shape of scores: (1, 300)
Shape of classes: (1, 300)
Shape of num_detections: (1,)
Target Class: 3
Gradient tensor: None
Gradient is None.
